<h1><strong>CSCI 4455/5455&ndash; Fall 2021</strong></h1>

<h2><strong>Assignment 3 - Frequent Pattern Mining</strong></h2>

<h3><strong><span style="color:#cc3300;">Due: November 19, 11:59pm </span></strong></h3>

<h3><strong>Your name:</strong></h3>

<ul>
<li style="text-align: justify;">Please note that you must do this assignment&nbsp;<span style="color: #cc3300;"><strong><u>individually</u></strong></span>. Using automatic tools, your code will be checked against other submissions and other existing resources (such as websites and books).</li>
<li style="text-align: justify;">This assignment is more extensive and might take longer than previous assignments to finish. <span style="color: #339966;"><strong><u>Please start early on.</u></strong></span></li>
<li style="text-align: justify;">Review the lecture notes before starting with this assignment. Then, thoroughly read this document before starting with the implementation or thinking about the solution.</li>
<li style="text-align: justify;">If you have technical questions about Python, please Google the error messages and share the error message alongside the solution that got it fixed on Microsoft Teams, as your classmates may run into the same issues.</li>
<li style="text-align: justify;">Check Canvas regularly for possible clarifications and updates.</li>
<li style="text-align: justify;">There are libraries and scripts for frequent pattern mining, but you are <span style="color: #cc3300;"><strong>prohibited to use these existing resources</strong></span>, which means you <span style="color: #cc3300;"><strong>cannot include public libraries</strong></span>, or <span style="color: #cc3300;"><strong>modify existing programs</strong></span> since the purpose   of   this   programming   assignment   is   to   help you understand and implement frequent pattern mining algorithms. You need to develop your code from scratch.</li>
</ul>


<h2><strong>Assignment Objectives</strong></h2>
<li style="text-align: justify;">1. To implement the <span style="color: #339966;"><strong>FP-Growth</strong></span> algorithm and test it under different configurations</li>
<li style="text-align: justify;">2. To mine maximal patterns</li>
<li style="text-align: justify;">3. To mine association rules from the frequent patterns</li>
<li style="text-align: justify;">4. To perform a sensitivity analysis on the minimum support threshold</li>

<h2>Dataset</h2>

<p style="text-align: justify;">A dataset is provided with this assignment that contains retail market basket data from an anonymous Belgian retail store. 
    
The retail dataset was originally used in the following paper: <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.67.3296&rep=rep1&type=pdf">paper link.</a>

    
Each line in the .txt file shows the items purchased in one transaction (transaction IDs are not provided), and items are separated by a space.    
</p>

<h2>Implementation</h2>

<p style="text-align: justify;">Please consider the following in your implementation:<br>
    
<ul>

<li style="text-align: justify;">You are not allowed to use frequent itemset mining libraries and need to implement your code from scratch. However, you are allowed to use Python built-in functions, such as min, max, average, map, apply, reduce, etc.</li>
<li style="text-align: justify;">Ensure that the cells in your Notebook are ordered correctly so that the “run all” option can run all cells without running to dependency issues.</li>
</ul>

<span style="text-align: justify;"><strong><u>Hint</u></strong> Running the code with the provided dataset may take a long time. Moreover, verifying the correctness of your code with the provided dataset is not easy since it contains many transactions. Therefore, you should use a small dataset with a few transactions during the implementation to test your code fast and verify the results. However, note that you may obtain correct results for a small dataset, even though your code may have bugs, so eventually, you should try your code with the larger dataset.
</span>
</p>

In [1]:
import matplotlib.pyplot as plt
import collections
import math
from itertools import combinations
import time
from collections import defaultdict

dataset_name =   'retail.txt'     # dataset_name: that is the dataset name (such as dataset.txt)
patterns_path =  'patterns.csv'   # the file name that will store the frequent patterns
rules_path =     'rules.csv'      # the file name that will store the maximal patterns
maximal_path =   'maximal.csv'    # the file name that will store the association rules
min_support =     0.0015            # minimuum support threshold which is in percentage format (for example 0.2 means 20%)
min_confidence =  0.9             # the minimum confidence threshold in the percentage format

datasetFile = open("project.dataset1.txt", "r")
listOfItems = []
frequency = []
for line in datasetFile:
    strippedLine = line.strip()
    lineList = strippedLine.split()
    listOfItems.append(lineList)
    frequency.append(1)

datasetFile.close()

numberOfTransaction = len(listOfItems)

<p>Note that in the rest of this Notebook, support count refers to the raw frequency (i.e., the number of times an itemsets occurs in a database), while support is the normalized support count which is equal to $\frac{\text{support count}}{\text{the number of transactions}}$ </p>

<h2>Implement the FP-Growth Algorithm (40 Points)</h2>

<p style="text-align: justify;">Implement the FP-Growth algorithm, as discussed in the lecture notes. Display the total execution time, the total number of generated candidates, and the total number of frequent patterns in the following format (the provided values are hypothetical):

    Execution time: 20 seconds
    Candidates: 20,021
    Frequent itemsets: 10,123
    
Each line must contain one pattern, its support count, and support. For example, the pattern A-B-C with support count 4 and support 0.5 must be saved in one line as A-B-C,4,0.5 in the <i><u>patterns.csv</u></i> file.
</p>

In [2]:
# wrtie the code to populate the transactions tidlist and implement the FP-Growth algorithm here
# you can insert more cells below, if needed
class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency
    def display(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.display(ind + 1)


def updateHeaderTable(item, targetNode, headerTable):
    if (headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]

def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if (len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    fpTree = Node('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    return fpTree, headerTable

def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)


def findPrefixPath(basePat, headerTable):
    # First node in linked list
    treeNode = headerTable[basePat][1]
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = [basePat]
        # From leaf node all the way to root
        ascendFPtree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next node
        treeNode = treeNode.next
    return condPats, frequency

def reportPatters(headerTable):
    df_patterns = pd.DataFrame(columns=['pattern','support_count','support'])
    candidate = 0
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p: p[1][0])]
    for item in sortedItemList:
        conditionalPattBase, frequency = findPrefixPath(item, headerTable)
        for patt in conditionalPattBase:
            candidate +=1
            idx = conditionalPattBase.index(patt)
            support_count = frequency[idx]
            support = frequency[idx]/numberOfTransaction
            if support >= min_support:
                row = {'pattern':patt, 
                       'support_count':support_count, 
                       'support':support}
                df_patterns = df_patterns.append(row,ignore_index=True)
    df_patterns.pattern = df_patterns.pattern.astype(str)
    df_patterns.pattern = df_patterns.pattern.str.replace("'","")
    df_patterns.pattern = df_patterns.pattern.str.replace("[","",regex=True)
    df_patterns.pattern = df_patterns.pattern.str.replace("]","",regex=True)
    df_patterns.pattern = df_patterns.pattern.str.replace(",","-",regex=True)
    df_patterns.pattern = df_patterns.pattern.str.replace(" ","",regex=True)
    frItemSet = df_patterns.shape[0]
    df_patterns.to_csv(patterns_path,index=False) 
    return frItemSet,candidate
  
startTime = time.time()
fpTree, headerTable = constructTree(listOfItems,frequency,min_support)
frItemSet, candidate = reportPatters(headerTable)
endTime = time.time()
print('Execution Time:',endTime - startTime)
print('Candidate:',candidate)
print('Frequent itemset:',frItemSet)

<h2>Find Maximal Itemsets (20 Points)</h2>

<p style="text-align: justify;">Find the maximal itemset among all the generated frequent patterns. You do not need to implement a maximal itemset mining algorithm (such as the Charm algorithm that allows for mining maximal itemsets without mining the frequent itemsets). Instead, you can simply iterate through the frequent itemsets and identify the maximal ones.

<p style="text-align: justify;">Finally, display the number of maximal patterns and the compression ratio in the following format (the provided values are hypothetical):

    Frequent itemsets: 10,000
    Maximal patterns: 1,500
    Compression ratio: 85%

As before, each line in your <i>maximal.csv</i> output file must contain one pattern, its support count, and support.
</p>


<p style="text-align: justify;"><strong>Hint</strong>: Iterate through the frequent itemsets with the reverse order that were mined in the previous step, i.e., from the longest itemset(s) to shorter one(s). </p>


In [3]:
# wrtie the code to implement and find maximal itemsets here
# you can insert more cells below, if needed
df_patterns = pd.read_csv(patterns_path)
df_maximal = pd.DataFrame(columns=['pattern','support_count','support'])
for index in df_patterns.index:
    if(df_patterns.pattern.apply(lambda y : df_patterns.pattern.apply(lambda x : set(x).issubset(y)))[index].sum() == 1):
        df_maximal = df_maximal.append([df_patterns.loc[index]],ignore_index=True)
print('Frequent itemsets:',df_patterns.shape[0])
print('Maximal patterns:',df_maximal.shape[0])
print('Compression Ratio:'+"{:.2%}".format(1-df_maximal.shape[0]/df_patterns.shape[0]))
df_maximal.to_csv(maximal_path,index=False)

<h2>Mine the Association Rules (30 Points)</h2>


<p style="text-align: justify;">Implement association rule mining to generate strong association rules from the frequent itemsets generated in Step 1 using the below algorithm. Display the confidence for the top 20 rules with the highest confidence. Moreover, report the execution time as well as the number of strong rules in the following format (the provided values are hypothetical): </p>
    
    Execution time: 12 secodns 
    Strong rules: 2,300
    Rule 1: A=>B,C, conf = 0.43
    Rule 2: X=>Y, conf = 0.40

Each line in the <i>rules.csv</i> file must contain one rule alongside its support count, support, and confidence. For example, A-B=>C,4,0.5,0.2 shows the support count, support, and confidence for the rule A-B=>C. 

In [4]:
# wrtie the code to implement and mine the association rules here
# you can insert more cells below, if needed
def find_rules(pattern_str):
    subset = []
    if len(pattern_str)<= 1:
        return None
    for i in range(1,len(pattern_str)):
        for l in set(combinations(pattern_str,i)):
            subset.append(''.join(l))
        pattern_dic = {}
        for pa in subset:
            pattern_dic[pa]=set(pattern_str).difference(set(pa))
    return pattern_dic

def store_rules():
    df_rules = pd.DataFrame(columns=['rules','support_count',
                                     'support','confidence'])
    support = lambda y : df_patterns.pattern.apply(lambda x : set(y).issubset(x)).sum()
    for idx in df_patterns.index:
        pattern_dic = find_rules(df_patterns.pattern[idx].replace('-',''))
        if pattern_dic != None:
            for key in pattern_dic.keys():
                left_side_support = support(key)
                confidence = df_patterns.support_count[idx]/left_side_support
                patt = str(key)+'->'+str(pattern_dic[key]).replace('{','').replace('}','').replace("'",'')
                row = {'rules':patt, 
                       'support_count':df_patterns.support_count[idx],
                       'support': df_patterns.support[idx],
                       'confidence':confidence}
                if confidence > min_confidence:
                    df_rules = df_rules.append(row,ignore_index=True)
    df_rules.to_csv(rules_path,index=False)
    return df_rules.shape[0]
        


startTime = time.time()
number_strong_rule = store_rules()
df_rules = pd.read_csv(rules_path)
df_rules.sort_values('confidence',inplace=True,ascending=False)
print(df_rules.head(20))
endTime = time.time()
print('Number of Strong Rules:',number_strong_rule)
print('Execution Time:',endTime - startTime)



<h2>Sensitivity Analysis (10 Points)</h2>


<p style="text-align: justify;">Run the frequent pattern mining function with the following values of minimum support and measure the total number of frequent patterns and generated candidates and the execution time. Next, draw three line plots using the matplotlib library that shows how minimum support (the x-axis) affects the runtime and the number of candidates and frequent itemsets (the y-axis). Note that x and y axes must have proper titles.
    
Minimum support values:
    
    0.0005, 0.001, 0.0015, 0.002, 0.0025, 0.003, 0.0035, 0.004, 0.0045, 0.005, 0.0055, 0.006, 0.0065, 0.007, 0.0075, 0.008, 0.0085, 0.009, 0.0095, 0.01


In [5]:
# wrtie the code here
# you can insert more cells below, if needed
minimum_support_values = [0.0005, 0.001, 0.0015, 0.002, 0.0025, 0.003,
                          0.0035, 0.004, 0.0045, 0.005, 0.0055, 0.006, 
                          0.0065, 0.007, 0.0075, 0.008, 0.0085, 0.009, 
                          0.0095, 0.01]

def display_plot():
    total_number_of_frequent_patterns = []
    generated_candidates = []
    execution_time = []
    for min_support in minimum_support_values:
        startTime = time.time()
        fpTree, headerTable = constructTree(listOfItems,frequency,min_support)
        frItemSet, candidate = reportPatters(headerTable)
        endTime = time.time()
        total_number_of_frequent_patterns.append(frItemSet)
        generated_candidates.append(candidate)
        execution_time.append(endTime-startTime)
    fig, ax = plt.subplots() 
    ax.plot(minimum_support_values,total_number_of_frequent_patterns, label='Total Number of Frequent Patterns') 
    ax.plot(minimum_support_values,generated_candidates, label='Generated Candidate') 
    ax.plot(minimum_support_values,execution_time, label='Execution Time')  # ... and some more.
    ax.set_xlabel('Minimum Support Value')
    ax.set_title("Sensitivity Analysis")  
    ax.legend()  

display_plot()

<h3>Coding Considerations</h3>
<p style="text-align: justify;">You must consider the following in your implementations:</p>
<ul>
<li class="a"><span> The output CSV files should be generated in the same directory of your code when you run it. Use the naming provided in the Implementation section. </span>
<li class="a"><span> You should write your code in this Jupyter Notebook (*.ipynb) file which can generate the required reports itself.</span>
<li class="a"><span> Your code should be self-explanatory. Make sure you add comments and your output files are formatted correctly. You might lose up to 30 points for bad code quality (readability, modularity, comments, efficiency, etc.) and formatting of the output files.</span></li>
</ul>

<h2>Submission</h2>
<p style="text-align: justify;">Your python file or Jupyter Notebook file must generate all the abovementioned reports when you run it.</p>
<p style="text-align: justify;">You need to submit a zip file in Canvas, including the following items: 

<ul>
<li class="a"><span> a Jupyter Notebook (*.ipynb) file, named assignment3.ipynb that contains your completed code </span>
<li class="a"><span> patterns.csv that contains the frequent patterns</span>
<li class="a"><span> maximal.csv that contains the maximal patterns</span>
<li class="a"><span> rules.csv that contains the association rules</span>
<li class="a"><span> results.pdf: a pdf file that contains all the requested charts and outputs (only copy the outputs (not the code itself) from your Notebook to this file. Also do not include the patterns and their supports in this file.)</span>
    
</li>
</ul>
    
<span style="background-color: #ffff99;">The file name should be in <strong>FirstName_LastName</strong> format</span>.</p>
<p style="text-align: justify;"><span style="background-color: #ffff99;">DO NOT INCLUDE EXTRA FILES, SUCH AS THE INPUT DATASETS</span>, in your submission;</p>
<p style="text-align: justify;">Please download your assignment after submission and make sure it is not corrupted or empty! We will not be responsible for corrupted submissions and will not take a resubmission after the deadline.</p>

<h2>Need Help?</h2>
<p>If you need help with this assignment, please get in touch with Erfan (on MS Teams or via email at <a href="mailto:erfan.jafarikhademzavareh@ucdenver.edu">Erfan.jafarikhademzavareh@ucdenver.edu</a>) or go to his office hours.</p>
<p>&nbsp;</p>
<p>You are highly encouraged to ask your question on the designated channel for Assignment 2 on Microsoft Teams (not necessarily monitored by the instructor/TA). Feel free to help other students with general questions. However, DO NOT share your solution.</p>